# Day 17

## part 1

- 3-bit computer with 3 registers, each which can hold any integer
- 8 instructions:
- instruction pointer starts at 0 and increments by 2 when not jumping
- program halts the counter moves past the end of the program
- operands can be literal or combo
    - literal gives the value itself
    - combo operands 0-3 are literals
    - combo operands 4-6 map to `A`, `B`, `C`
    - combo operand 7 is unot used

Here are the opcodes

|code|name|function|stored to|
|---|---|---|---|
|0|adv|divides the `A` register by $2^{x}$ where x is the combo operand, truncated to an int|`A`|
|1|bxl|bitwise XOR of register B and the literal operand|`B`|
|2|bst|combo operand % 8|`B`|
|3|jnz|nop if `A`==0. If `A`!=0 jumps to literal operand and instruction pointer is not incremented this time| |
|4|bxc|bitwise XOR of `B` and `C`|`B`|
|5|out|combo operand % 8|output|
|6|bdv|as adv but for `B`|`B`|
|7|cdv|as adv but for `C`|`C`|


- Determine the program output as comma delimited values


In [46]:
from dataclasses import dataclass
import logging

from advent_of_code_utils.advent_of_code_utils import (
    parse_from_file, ParseConfig as PC, markdown
)

log = logging.getLogger('day 17')
logging.basicConfig(level=logging.INFO)

In [47]:
@dataclass
class Computer:
    A: int
    B: int
    C: int
    programme: list[int]
    pointer: int = 0

    def step(self) -> list[int] | None:
        """runs the next programme step"""
        log.debug(f'{self}')
        if self.pointer >= len(self.programme):
            log.info('halt')
            return None
        opcode = self.programme[self.pointer]
        operand = self.programme[self.pointer + 1]
        log.debug(f'{opcode=}, {operand=}')
        out = []
        match opcode:
            case 0:
                self.adv(operand)
                self.increment()
            case 1:
                self.bxl(operand)
                self.increment()
            case 2:
                self.bst(operand)
                self.increment()
            case 3:
                self.jnz(operand)
            case 4:
                self.bxc(operand)
                self.increment()
            case 5:
                out.append(self.out(operand))
                self.increment()
            case 6:
                self.bdv(operand)
                self.increment()
            case 7:
                self.cdv(operand)
                self.increment()
            case _:
                raise ValueError(f'opcode not recognised: {self}')
        return out
    
    def increment(self) -> None:
        """increments the programme pointer"""
        self.pointer += 2
        log.debug('pointer: +2')

    def combo(self, operand: int) -> int:
        """returns the combo operand value"""
        reg_map = {4: self.A, 5: self.B, 6: self.C}
        if operand in [0, 1, 2, 3]:
            return operand
        elif operand in reg_map:
            return reg_map[operand]
        else:
            raise ValueError(f'Invalid combo operand: {self}')

    def adv(self, operand: int) -> None:
        numerator = self.A
        denominator = pow(2, self.combo(operand))
        self.A = numerator // denominator
        log.debug(f'adv: {self.A}->A')
    
    def bdv(self, operand: int) -> None:
        numerator = self.A
        denominator = pow(2, self.combo(operand))
        self.B = numerator // denominator
        log.debug(f'bdv: {self.B}->B')
    
    def cdv(self, operand: int) -> None:
        numerator = self.A
        denominator = pow(2, self.combo(operand))
        self.C = numerator // denominator
        log.debug(f'cdv: {self.C}->C')

    def bxl(self, operand: int) -> None:
        self.B ^= operand
        log.debug(f'bxl: {self.B}->B')
    
    def out(self, operand: int) -> int:
        value = self.combo(operand) % 8
        log.debug(f'out: {value}')
        return value
    
    def bst(self, operand: int) -> None:
        self.B = self.combo(operand) % 8
        log.debug(f'bst: {self.B}->B')
    
    def jnz(self, operand: int) -> None:
        if self.A == 0:
            log.debug('jnz: A=0')
            self.increment()
        else:
            log.debug(f'jnz: A!=0, pointer->{operand}')
            self.pointer = operand
    
    def bxc(self, *args) -> None:
        self.B ^= self.C
        log.debug(f'bxc: {self.B}->B')

parser = PC('\n\n', [
    PC('\n', PC(': ', [None, int])),  # registers
    PC(': ', [None, PC(',', int)])  # program
])
registers, programme = \
    parse_from_file('day_17_example.txt', parser, unnest_single_items=True)
computer = Computer(*registers, programme)

INFO:advent_of_code_utils.py:2 items loaded from "day_17_example.txt"


In [48]:
log.setLevel(logging.INFO)
log.debug(f'{computer=}')
output = []
while True:
    temp = computer.step()
    if temp is None:
        break
    else:
        output.extend(temp)
log.info(f'{output=}')

INFO:day 17:halt
INFO:day 17:output=[4, 6, 3, 5, 6, 3, 5, 2, 1, 0]


In [51]:
# great that looks like it works, lets run things for real
log.setLevel(logging.INFO)
registers, programme = \
    parse_from_file('day_17.txt', parser, unnest_single_items=True)
computer = Computer(*registers, programme)
output = []
while True:
    temp = computer.step()
    if temp is None:
        break
    else:
        output.extend(temp)

INFO:advent_of_code_utils.py:2 items loaded from "day_17.txt"
INFO:day 17:halt


In [50]:
markdown(f'The programme output is: {','.join((str(v) for v in output))}')

The programme output is: 4,6,1,4,2,1,3,1,6